In [ ]:
import os
import subprocess

from operator import or_
from pprint import pprint
from functools import reduce

In [ ]:
import pandas as pd
import ujson

In [ ]:
from conda.cli.python_api import run_command, Commands
from conda.history import History

In [ ]:
def get_all_conda_envs():
    '''return all conda environment paths excluding base/root

    using conda's Python API
    '''
    return [
        env
        for env in ujson.loads(run_command(Commands.INFO, '--json')[0])['envs']
        if '/envs/' in env # filter out the root/base env
    ]

In [ ]:
def conda_list(env):
    '''return a DataFrame representation of ``conda list`` for environment at `env`

    this one uses the conda Python API that excludes pip installed packages
    '''
    temp = ujson.loads(run_command(Commands.LIST, '--prefix', env, '--json')[0])
    df = pd.DataFrame(temp)
    df.set_index('name', inplace=True)
    return df

In [ ]:
def conda_list_subprocess(env):
    '''return a DataFrame representation of ``conda list`` for environment at `env`

    this one uses the cli directly that includes pip installed packages
    '''
    temp = ujson.loads(subprocess.run(('conda', 'list', '--prefix', env, '--json'), stdout=subprocess.PIPE).stdout)
    df = pd.DataFrame(temp)
    df.set_index('name', inplace=True)
    return df

In [ ]:
def get_user_installed_packages(env):
    '''return user installed packages in prefix `env`

    using undocumented conda Python API, see
    https://github.com/conda/conda/issues/4545#issuecomment-469984684
    '''
    history = History(env)
    return history.get_requested_specs_map().keys()

In [ ]:
def filter_channels(env, channels=('pypi',)):
    '''return packages from `channels` in environment `env`
    '''
    df = conda_list_subprocess(env)
    return df[df.channel.isin(channels)].index

In [ ]:
def remove_element(list_, elements):
    '''filter out items in `list_` that contains element in `elements`
    '''
    return [
        item for item in list_
        if not reduce(
            or_,
            (element in item for element in elements)
        )
    ]

In [ ]:
def map_union(func, iterables):
    '''set union of the results from `func` applied to items in `iterables`
    '''
    return set().union(*[set(func(item)) for item in iterables])

In [ ]:
envs = remove_element(get_all_conda_envs(), ('jupyterlab', 'ISR', 'sage'))

In [ ]:
envs

# Conda

In [ ]:
# set of all user installed packages in envs
result = map_union(get_user_installed_packages, envs)

In [ ]:
with open(os.path.expanduser('~/git/source/reproducible-os-environments/common/conda/conda-all.txt'), 'r') as f:
    txt_all = [word.lstrip('#').strip() for word in f.readlines() if word.lstrip('#').strip()]

In [ ]:
with open(os.path.expanduser('~/git/source/reproducible-os-environments/common/conda/conda.txt'), 'r') as f:
    txt = [word.lstrip('#').strip() for word in f.readlines() if word.lstrip('#').strip()]

In [ ]:
# user installed packages not in conda-all.txt
sorted(i for i in result if i not in txt_all)

In [ ]:
# packages in conda.txt not in conda-all.txt
sorted(i for i in txt if i not in txt_all)

# pip

In [ ]:
# all pypi packages from envs
pip_packages = map_union(filter_channels, envs)

In [ ]:
with open(os.path.expanduser('~/git/source/reproducible-os-environments/common/conda/pip-all.txt'), 'r') as f:
    txt_all = [word.lstrip('#').strip() for word in f.readlines() if word.lstrip('#').strip()]

In [ ]:
with open(os.path.expanduser('~/git/source/reproducible-os-environments/common/conda/pip.txt'), 'r') as f:
    txt = [word.lstrip('#').strip() for word in f.readlines() if word.lstrip('#').strip()]

In [ ]:
# pypi packages not in pip-all.txt
sorted(i for i in pip_packages if i not in txt_all)

In [ ]:
# packages in pip.txt not in pip-all.txt
sorted(i for i in txt if i not in txt_all)